In [15]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output
import plotly.express as px

In [ ]:
# dataset/Cleaned_Census_Dataset.csv
df = pd.read_csv('dataset/Cleaned_Census_Dataset.csv',dtype={'NOC': str}) 
df['NOC'] = df['NOC'].astype(str)

population_data = {
    'Newfoundland and Labrador': 525000,
    'Prince Edward Island': 175000,
    'Nova Scotia': 1000000,
    'New Brunswick': 800000,
    'Quebec': 8700000,
    'Ontario': 14900000,
    'Manitoba': 1400000,
    'Saskatchewan': 1200000,
    'Alberta': 4600000,
    'British Columbia': 5400000,
    'Yukon': 43000,
    'Northwest Territories': 45000,
    'Nunavut': 40000
}

population_df = pd.DataFrame(list(population_data.items()), columns=['Province', 'Population'])
df = df.merge(population_df, on='Province')
df['Employment_per_10k'] = (df['Employment'] / df['Population']) * 10000
df.head(20)



,Province,Occupation,NOC,Gender,Employment,Population,Employment_per_10k
0,Newfoundland and Labrador,Occupation - not applicable 13,nan,Men,74715.0,525000,1423.142857
1,Newfoundland and Labrador,All occupations 14,nan,Men,136465.0,525000,2599.333333
2,Newfoundland and Labrador,0 Legislative and senior management occupations,0,Men,1270.0,525000,24.190476
3,Newfoundland and Labrador,00 Legislative and senior managers,00,Men,1270.0,525000,24.190476
4,Newfoundland and Labrador,000 Legislative and senior managers,000,Men,1270.0,525000,24.190476
5,Newfoundland and Labrador,0001 Legislative and senior managers,0001,Men,1270.0,525000,24.190476
6,Newfoundland and Labrador,00010 Legislators,0001,Men,25.0,525000,0.476190
7,Newfoundland and Labrador,00018 Seniors managers - public and private se...,0001,Men,1245.0,525000,23.714286
8,Newfoundland and Labrador,"1 Business, finance and administration occupat...",1,Men,9400.0,525000,179.047619
9,Newfoundland and Labrador,10 Specialized middle management occupations i...,10,Men,1505.0,525000,28.666667


In [ ]:
app = dash.Dash(__name__)
app.title = "2023 Canadian Census Dashboard"

app.layout = html.Div([
    html.H1("2023 Canadian Census - Employment Dashboard", style={'color': 'lightblue'}),

    html.Div([
        html.Label("Select Essential Service", style={'color': 'lightblue'}),
        dcc.Dropdown(
            id='essential-service-dropdown',
            options=[
                {'label': 'Registered Nurses', 'value': 'Registered nurses and registered psychiatric nurses'},
                {'label': 'Commissioned Police Officers', 'value': 'Commissioned police officers and related occupations in public protection services'},
                {'label': 'Fire Chiefs and Senior Officers', 'value': 'Fire chiefs and senior firefighting officers'},
            ],
            value='Registered nurses and registered psychiatric nurses'
        ),
    ]),

    dcc.Graph(id='essential-service-graph'),

    html.Div([
        html.Label("Select Province for Gender vs NOC Analysis", style={'color': 'lightblue'}),
        dcc.Dropdown(
            id='province-dropdown',
            options=[{'label': province, 'value': province} for province in df['Province'].unique()],
            value='Ontario'
        )
    ]),

    dcc.Graph(id='gender-noc-graph'),

    html.Div([
        html.Label("Select Province for EV Engineer Availability", style={'color': 'lightblue'}),
        dcc.Dropdown(
            id='ev-province-dropdown',
            options=[{'label': province, 'value': province} for province in df['Province'].unique()],
            value='Ontario'
        )
    ]),

    dcc.Graph(id='ev-engineer-graph'),

    html.Div([
        html.Label("Select a Gender to Explore Open Insight", style={'color': 'lightblue'}),
        dcc.Dropdown(
            id='open-gender-dropdown',
            options=[
                {'label': 'Men', 'value': 'Men'},
                {'label': 'Women', 'value': 'Women'}
            ],
            value='Women'
        )
    ]),

    dcc.Graph(id='open-ended-graph')
])


@app.callback(
    Output('essential-service-graph', 'figure'),
    Input('essential-service-dropdown', 'value')
)
def update_essential_service_chart(selected_occupation):
    dff = df[df['Occupation'].str.contains(selected_occupation, case=False, na=False)]
    dff_grouped = dff.groupby(['Province'], as_index=False)['Employment_per_10k'].sum()
    fig = px.bar(
        dff_grouped,
        x='Province',
        y='Employment_per_10k',
        title=f"Essential Service Availability - {selected_occupation} (per 10,000)",
        labels={'Employment_per_10k': 'Employment per 10,000'}
    )
    return fig

@app.callback(
    Output('gender-noc-graph', 'figure'),
    Input('province-dropdown', 'value')
)
def update_gender_noc_chart(selected_province):
    dff = df[(df['Province'] == selected_province) & ((df['NOC'] == '1') | (df['NOC'] == '2'))]
    dff_grouped = dff.groupby(['Gender', 'NOC'], as_index=False)['Employment_per_10k'].sum()
    noc_labels = {'1': 'Business (NOC 1)', '2': 'STEM (NOC 2)'}
    dff_grouped['Occupation Group'] = dff_grouped['NOC'].map(noc_labels)

    fig = px.funnel(
        dff_grouped,
        x='Employment_per_10k',
        y='Occupation Group',
        color='Gender',
        title=f"Gender Distribution in Business and STEM Occupations in {selected_province}"
    )
    return fig

@app.callback(
    Output('ev-engineer-graph', 'figure'),
    Input('ev-province-dropdown', 'value')
)
def update_ev_chart(selected_province):
    engineer_titles = [
        '21301 Mechanical engineers',
        '21310 Electrical and electronics engineers',
        '21311 Computer engineers (except software engineers and designers)',
        '2131 Electrical, electronics and computer engineers'
    ]
    dff = df[df['Province'] == selected_province]
    dff = dff[df['Occupation'].isin(engineer_titles)]
    dff_grouped = dff.groupby(['Occupation'], as_index=False)['Employment_per_10k'].sum()

    fig = px.line_polar(
        dff_grouped,
        r='Employment_per_10k',
        theta='Occupation',
        line_close=True,
        title=f"Engineer Workforce for EV Factory Setup in {selected_province}"
    )
    return fig

@app.callback(
    Output('open-ended-graph', 'figure'),
    Input('open-gender-dropdown', 'value')
)
def update_open_insight_chart(selected_gender):
    dff = df[(df['Gender'] == selected_gender)]
    dff_grouped = dff.groupby('Province', as_index=False)['Employment_per_10k'].sum()

    if dff_grouped.empty:
        fig = px.line(title="No data available for the selected gender.")
    else:
        fig = px.pie(
            dff_grouped,
            names='Province',
            values='Employment_per_10k',
            title=f"Proportional Employment per Province for {selected_gender}"
        )
    return fig

if __name__ == '__main__':
    app.run(debug=True)


/var/folders/vh/zss70z7x4llcd4_hh7j1m5xr0000gn/T/ipykernel_31691/3718740872.py:107: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/vh/zss70z7x4llcd4_hh7j1m5xr0000gn/T/ipykernel_31691/3718740872.py:107: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/vh/zss70z7x4llcd4_hh7j1m5xr0000gn/T/ipykernel_31691/3718740872.py:107: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/vh/zss70z7x4llcd4_hh7j1m5xr0000gn/T/ipykernel_31691/3718740872.py:107: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/vh/zss70z7x4llcd4_hh7j1m5xr0000gn/T/ipykernel_31691/3718740872.py:107: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/var/folders/vh/zss70z7x4llcd4_hh7j1m5xr0000gn/T/ipykernel_31691/3718740872.py:107: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

